In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Part I: read data

In [141]:
## try first 50
train = pd.read_csv("train.csv").head(100)
test = pd.read_csv("test.csv").head(100)

In [4]:
train.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count'],
      dtype='object')

## bag of words

In [142]:
X = train
y = train['target']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 0)
y_train_bi = y_train>0.5
y_valid_bi = y_valid>0.5
X_test = test['comment_text']

In [66]:
bow = CountVectorizer(stop_words = "english", ngram_range=(1, 2),binary = True)
tfidf = TfidfVectorizer(stop_words = "english", ngram_range=(1, 2),binary = True)

In [67]:
bow.fit(X_train,y_train)
X_train_bow = bow.transform(X_train)
X_valid_bow = bow.transform(X_valid)

In [68]:
tfidf.fit(X_train,y_train)
X_train_tfidf = tfidf.transform(X_train)
X_valid_tfidf = tfidf.transform(X_valid)

### Logistic Regression

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve,auc

In [57]:
lr_bow = LogisticRegression(solver = "lbfgs").fit(X_train_bow,y_train_bi)
y_score_lr_bow = lr_bow.decision_function(X_valid_bow)
lr_bow_fpr,lr_bow_tpr, _ = roc_curve(y_valid_bi,y_score_lr_bow)
lr_bow_roc_auc = auc(lr_bow_fpr,lr_bow_tpr)
lr_bow_roc_auc

1.0

## embedding

In [143]:
raw_text_train = X_train.loc[:,"comment_text"]
raw_text_valid = X_valid.loc[:,"comment_text"]
raw_text_test = test.loc[:,"comment_text"]

tk = Tokenizer()
tk.fit_on_texts(raw_text_train)

X_train.loc[:,"comment_seq"] = tk.texts_to_sequences(raw_text_train)
X_valid.loc[:,"comment_seq"]= tk.texts_to_sequences(raw_text_valid)
test.loc[:,"comment_seq"] = tk.texts_to_sequences(raw_text_test)

X_train_s = pad_sequences(X_train.comment_seq)
X_valid_s = pad_sequences(X_valid.comment_seq)
test_s = pad_sequences(test.comment_seq)